In [ ]:
import os
from glob import glob
import wfdb
import numpy as np
import pandas as pd
import scipy

In [ ]:
root = "/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/Datasets"
new_data_folder = "PTB_Diagnostic_data"
datapath = os.path.join(root , new_data_folder)

In [ ]:
data_records = []

for folder in os.listdir(datapath):
    if folder == "CONTROLS":
        continue
    patient_folder = os.path.join(datapath, folder)
    records = glob(os.path.join(patient_folder , "*.hea"))
    for record in records:
        data_records.append([patient_folder.split('/')[-1]])
        data_records[-1].append(record.split('/')[-1][:-4])
        data , info = wfdb.io.rdsamp(os.path.join(record[:-4]))
        sample_rate = info['fs']
        signal_len = info['sig_len']
        num_signals = info['n_sig']
        signal_order = info['sig_name']
        age = info['comments'][0][5:]
        age = int(age) if age != "n/a" else np.NaN
        sex = info['comments'][1][5:]
        dx = info['comments'][4][len("reason for admission: "):]
        data_records[-1].append(sample_rate)
        data_records[-1].append(signal_len)
        data_records[-1].append(num_signals)
        data_records[-1].append(signal_order)
        data_records[-1].append(age)
        data_records[-1].append(sex)
        data_records[-1].append(dx)

    column_names = ['patient_id', 'record_number', 'sample_rate', 'signal_len', 'number_signals', 'signal_order' , 'age' , 'sex' , 'dx']
    df = pd.DataFrame(data=data_records, columns=column_names)
    df.sort_values('patient_id').to_csv("/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/ECG_Project/PTB_reference.csv", index=None)

In [64]:
os.mkdir("/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/Datasets/Processed_PTB_data")
os.chdir("/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/Datasets/Processed_PTB_data")
for folder in os.listdir(datapath):
    if folder == "CONTROLS":
        continue
    patient_folder = os.path.join(datapath, folder)
    records = glob(os.path.join(patient_folder , "*.hea"))
    for record in records:
        data , _ = wfdb.io.rdsamp(os.path.join(record[:-4]))
        dict  = {}
        dict["val"] = data
        file_name = patient_folder.split('/')[-1] + "_" + record.split('/')[-1][:-4]
        scipy.io.savemat(file_name=file_name + ".mat" , mdict=dict)